### Jul AdventKalender D14

https://adventofcode.com/2022/day/14

In [1]:
import numpy as np

#### Day 14.1 

Scan traces the path of each solid rock (x represents distance to the right and y represents distance down). For example:

    498,4 -> 498,6 -> 496,6
    503,4 -> 502,4 -> 502,9 -> 494,9
    
This becomes (image the sand/+ pour down from **500,0**):

      4     5  5
      9     0  0
      4     0  3
    0 ......+...
    1 ..........
    2 ..........
    3 ..........
    4 ....#...##
    5 ....#...#.
    6 ..###...#.
    7 ........#.
    8 ........#.
    9 #########.

Then simulate the sand falling. Sand is produced **one unit at a time**, and the next unit of sand is not produced until the previous unit of sand **comes to rest**. Sand unit goes **down > down-left > down-right** and comes to rest when none of the three positions are available. Sand **stops generate** when no more sand can possibly come to rest (goes into the endless void at the bottom). The above example will eventually look like as below, with 24 units of sand being poured down:

    ......+...
    ..........
    ......o...
    .....ooo..
    ....#ooo##
    ...o#ooo#.
    ..###ooo#.
    ....oooo#.
    .o.ooooo#.
    #########.

How many units of sand come to rest before sand starts flowing into the abyss below?

In [2]:
def readMap(file_name):
    data_rocks = []
    f = open(file_name, "r")
    while True:
        line = f.readline()
        if not line:
            break
        rock_steps = [[int(pos.strip().split(',')[0]),int(pos.strip().split(',')[1])] for pos in line.strip().split('->')]
        data_rocks.append(rock_steps)
    f.close()
    return data_rocks

In [3]:
# get map size
def getMapSize(data_rocks):
    left_right_down = [500,500,0]
    for path in data_rocks:
        for pos in path:
            left_right_down[0] = pos[0] if pos[0] < left_right_down[0] else left_right_down[0]
            left_right_down[1] = pos[0] if pos[0] > left_right_down[1] else left_right_down[1]
            left_right_down[2] = pos[1] if pos[1] > left_right_down[2] else left_right_down[2]
    return left_right_down

In [4]:
# generate map
def generateMap(width, height):
    return np.full((height, width), '.')

def drawMap(map_sim):
    print('\n'.join([''.join(row) for row in map_sim]))
    print('\n')

In [5]:
def _genList(start, end, ref = 0):
    res = range(start-ref,end-ref+1,1) if start <= end else range(end-ref, start-ref+1, 1)
    return list(res)

def markRocks(rock_path, map_sim, left_s):
    for i in range(1,len(rock_path),1):
        pos_cur = rock_path[i]
        pos_pre = rock_path[i-1]
        map_sim[_genList(pos_pre[1], pos_cur[1]), _genList(pos_pre[0], pos_cur[0], left_s)] = '#'

In [6]:
def pourSand(pos_s, map_sim):
    pos_sand = np.array(pos_s)
    while pos_sand[0]+1 < map_sim.shape[0]:
        if map_sim[pos_sand[0]+1,pos_sand[1]] == '.':
            pos_sand[0] += 1
        elif map_sim[pos_sand[0]+1,pos_sand[1]-1] == '.':
            pos_sand = pos_sand + np.array([1,-1])
        elif map_sim[pos_sand[0]+1,pos_sand[1]+1] == '.':
            pos_sand += 1
        else:
            map_sim[pos_sand[0],pos_sand[1]] = 'o'
            return True, pos_sand
    return False, None

In [7]:
data_rocks = readMap("data/input14.txt")

# generate map
left_right_down = getMapSize(data_rocks)
map_sim = generateMap(left_right_down[1]-left_right_down[0]+1, left_right_down[2]+1)
#drawMap(map_sim)

# scan rocks
for path in data_rocks:
    markRocks(path, map_sim, left_right_down[0])
#drawMap(map_sim)

# simulate sand
n_sand = 0
pos_sand_start = [0,500]
pos_sand_start[1] -= left_right_down[0]
while True:
    if_poured,_ = pourSand(pos_sand_start, map_sim)
    if not if_poured: break
    n_sand += 1
#drawMap(map_sim)
print('Number of poured sand:', n_sand)

Number of poured sand: 1298


#### Day 14.2

Now there isn't an endless void at the bottom of the scan - there's floor with an **infinite horizontal line** and a y coordinate equal to **two plus the highest y** in your scan. In the example above, the highest y coordinate of any point is 9, and so the floor is at y=11. 

Now the sand stops generate until a sand comes to rest at the position where the sand generate (500,0). The example above looks like the following now with a total of 93 units come to rest:

    ............o............
    ...........ooo...........
    ..........ooooo..........
    .........ooooooo.........
    ........oo#ooo##o........
    .......ooo#ooo#ooo.......
    ......oo###ooo#oooo......
    .....oooo.oooo#ooooo.....
    ....oooooooooo#oooooo....
    ...ooo#########ooooooo...
    ..ooooo.......ooooooooo..
    #########################
   
How many units of sand come to rest now?

In [8]:
data_rocks = readMap("data/input14.txt")

# add floor as rocks
left_right_down = getMapSize(data_rocks)
floor_append_length = 150 # should be large enough
floor = [[left_right_down[0]-floor_append_length, left_right_down[2]+2],[left_right_down[1]+floor_append_length, left_right_down[2]+2]]
data_rocks.append(floor)

# generate map
left_right_down = getMapSize(data_rocks)
map_sim = generateMap(left_right_down[1]-left_right_down[0]+1, left_right_down[2]+1)
#drawMap(map_sim)

# scan rocks
for path in data_rocks:
    markRocks(path, map_sim, left_right_down[0])
#drawMap(map_sim)

# simulate sand
n_sand = 0
pos_sand_start = [0,500]
pos_sand_start[1] -= left_right_down[0]
while True:
    if_poured, pos_poured = pourSand(pos_sand_start, map_sim)
    if not if_poured: break
    n_sand += 1
    if (pos_poured == np.array(pos_sand_start)).all(): break
#drawMap(map_sim)
print('Number of poured sand:', n_sand)

Number of poured sand: 25585
